In [46]:
import pandas as pd
import numpy as np
import geopy
from geopy.geocoders import Nominatim

In [47]:
df = pd.read_csv('tweet_sample.csv')

## Subset to 10 tweet coordinates for sake of Geopy Example

In [142]:
df_coordinates = df['coordinates'][:10]

In [75]:
df_coordinates

0           27.79571925, -82.67290299999999
1    38.003375500000004, -79.42086499999999
2            29.8384948, -95.44648649999999
3                    42.3527865, -83.099288
4            36.1395275, -85.51020700000001
5                  34.0207895, -118.4119065
6                34.16825365, -111.93171085
7           35.6565515, -105.99910650000001
8                    34.181956, -119.165389
9             41.699816, -86.07852399999999
Name: coordinates, dtype: object

## Convert Geocode Helper Files to Dictionaries

In [138]:
state_to_code_df = pd.read_csv('state_to_code.csv', header=None)
state_to_code_dict = {}
for row in state_to_code_df.itertuples():
    state_to_code_dict[row[1]] = row[2]

county_to_code_df = pd.read_csv('code_county_state.csv')
county_to_code_dict = {}
for row in county_to_code_df.itertuples():
    if len(str(row[1])) == 4: 
        code = '0' + str(row[1])
    else:
        code = str(row[1])
    county_to_code_dict[(str(row[2]),row[3])] = code

code_to_adi_df = pd.read_csv('county_adi.txt', dtype=str)
code_to_adi_dict = {}
for row in code_to_adi_df.itertuples():
    if str(row[2])[:5] not in code_to_adi_dict:
         code_to_adi_dict[str(row[2])[:5]] = []
    try:
        a = int(row[3])
    except:
        continue

    code_to_adi_dict[str(row[2])[:5]].append(int(row[3]))

## Reverse Geocode Function

In [ ]:
def coord_city_state(x):
    try:
        location = locator.reverse(x)
        if 'error' in location.raw:
            return ["NA", "NA", "NA", "NA", "NA"]
    except:
         return ["NA", "NA", "NA", "NA", "NA"]

    if 'county' in location.raw['address']:
        county = location.raw['address']['county']
    else:
        county = "NA"
    
    if 'state' in location.raw['address']:
        state = location.raw['address']['state']
    else:
        state = "NA"
    
    if county != "NA" and state != "NA":
        try:
            county_code = county_to_code_dict[(county, state)]
        except KeyError:
            county_code = "NA"

        try:
            if code_to_adi_dict[county_code]:
                adi = str(np.median(code_to_adi_dict[county_code]))
            else:
                adi = "NA"
        except KeyError:
            adi = "NA"

    else:
        county_code = "NA"
        adi = "NA"

    return [county, state, county_code, adi]

## Run Function

In [143]:
locator = Nominatim(user_agent="my-geopy-application")
df_coordinates['County-State-Code-ADI'] = df_coordinates.apply(lambda x:coord_city_state(x))

## Results

In [144]:
df_coordinates['County-State-Code-ADI']

0          [Pinellas County, Florida, 12103, 61.0]
1       [Rockbridge County, Virginia, 51163, 47.5]
2              [Harris County, Texas, 48201, 62.0]
3            [Wayne County, Michigan, 26163, 86.0]
4          [Putnam County, Tennessee, 47141, 60.5]
5    [Los Angeles County, California, 06037, 19.0]
6           [Yavapai County, Arizona, 04025, 53.0]
7       [Santa Fe County, New Mexico, 35049, 29.5]
8        [Ventura County, California, 06111, 13.0]
9      [Saint Joseph County, Indiana, 18141, 74.0]
Name: coordinates, dtype: object